In [1]:
import spacy
import numpy as np
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load("en_core_web_sm") 
nlp.add_pipe('spacytextblob')
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.svm import SVC 
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression

In [2]:
STOP_WORDS = {
    'i', 'me', 'my', 'myself', 'it', 'its', 'itself', 'what', 'which', 'who', 'whom', 'am', 'is', 'are',
    'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an',
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',
    'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up',
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when',
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only',
    'own', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
}

hate_speech_words = [
    'nigger', 'nigga', 'coon', 'spic', 'chink', 'kike', 'gook', 'wop', 'towelhead', 'slope',
    'kafir', 'infidel', 'heathen', 'idolater', 'heretic', 'blasphemer',
    'bitch', 'slut', 'whore', 'cunt', 'feminazi', 'tranny', 'dyke', 'faggot', 'fag', 'pussy', 'mansplainer',
    'faggot', 'fag', 'dyke', 'homo', 'queer', 'sissy', 'tranny', 'butch',
    'tranny', 'shemale', 'it', 'transvestite', 'gender bender', 'freak',
    'retard', 'spaz', 'cripple', 'invalid', 'lame', 'imbecile', 'moron', 'idiot',
    'fucktard', 'asshat', 'retard', 'douchebag', 'assclown', 'twat', 'twatwaffle', 'dickwad'
]

swear_words = [
    'fuck', 'fucking', 'fucked', 'fuckwit', 'fucktard', 'fuckface', 'fuckhead', 'motherfucker', 'shit', 'shitty', 'shite', 'shitting', 
    'shithead', 'shitstorm', 'ass', 'asshole', 'asshat', 'asswipe', 'dumbass', 'bitch', 'bitchy', 'son of a bitch',
    'bastard', 'damn', 'damned', 'dick', 'dickhead', 'dickwad', 'pussy', 'pussies', 'cock', 'cocksucker', 'cockhead',
    'cunt', 'cunty', 'cuntface', 'twat', 'twatwaffle', 'wanker', 'wank', 'wankstain', 'bollocks', 'douchebag', 'douche', 'douchecanoe',
    'arse', 'arsehole', 'arsehat', 'arsewipe', 'shitfuck', 'fuckshit', 'clusterfuck', 'fucknugget', 'fuckery',
    'cockwomble', 'cockgobbler', 'cockjockey', 'dickcheese', 'dicknose', 'shitlord', 'assclown', 'assjacket'
]



In [3]:
def preprocessing(sentences):
      rets=[]
      polarity=[]
      subjectivity=[]
      hate_count=[]
      for sent in sentences:
            sent = sent.replace("n't", " not")
            sent = sent.lower().strip()
            temp_arr= sent.split(" ")
            count=0
            for word in temp_arr:
                  if(word in hate_speech_words or word in swear_words):
                        count+=1
            hate_count.append(count/len(temp_arr))
            doc = nlp(sent)
            arr = [token.lemma_ for token in doc if (token.lemma_.lower() not in STOP_WORDS) and not token.is_punct and not token.like_num 
                  and not token.is_currency and not token.is_digit]
            arr= " ".join(arr)
            rets.append(arr)
            polarity.append(doc._.blob.polarity)
            subjectivity.append(doc._.blob.subjectivity)
      return rets, polarity, subjectivity, hate_count

In [4]:
#using caption generating model- BLIP, and using the provided jsonl files, the following table has been created
#it contains caption of image, description of the image generated by BLIP
caption_description_df= pd.read_pickle("../commons/caption_description.pkl")
caption_description_df.head(5)

,label,text,file,description,sets
0,0,berserk 2016 is a good adaptation you're kidd...,71094.png,woman with a monkey mask and a fake monkey,[test_unseen]
1,0,my life goal? make somebody this fucking trig...,91724.png,woman holding a cigarette in her hand,[train]
2,0,""" i don't wanna, just get it, get it, get it, ...",64280.png,man wearing a hat and a tie,[train]
3,0,"""1st day of 4th grade sandy hook elementary sc...",67082.png,group of children standing in front of a schoo...,"[dev_seen, dev_unseen]"
4,0,"""a blow job a day will keep his side chicks aw...",46380.png,woman with a black top and a blue background,[train]


In [5]:
# #Uncomment if necessary

# caption_df= caption_description_df[["label", "text", "file", "sets"]]
# rets, polarity, subjectivity, hate_count= preprocessing(caption_df["text"])
# caption_df["preprocessed text"]=rets
# caption_df["polarity"]=polarity
# caption_df["subjectivity"]=subjectivity
# caption_df["hate count"]= hate_count
# pd.to_pickle(caption_df, "../commons/caption_preprocessed.pkl")

caption_df= pd.read_pickle("../commons/caption_preprocessed.pkl")
caption_df.head()

,label,text,file,sets,preprocessed text,polarity,subjectivity,hate count
0,0,berserk 2016 is a good adaptation you're kidd...,71094.png,[test_unseen],berserk good adaptation you kid right,0.492857,0.567857,0.000
1,0,my life goal? make somebody this fucking trig...,91724.png,[train],life goal make somebody this fucking trigger,-0.600000,0.800000,0.125
2,0,""" i don't wanna, just get it, get it, get it, ...",64280.png,[train],not wanna get get get get that shit hard chanc...,-0.163889,0.480556,0.080
3,0,"""1st day of 4th grade sandy hook elementary sc...",67082.png,"[dev_seen, dev_unseen]",day grade sandy hook elementary school,0.300000,0.900000,0.000
4,0,"""a blow job a day will keep his side chicks aw...",46380.png,[train],blow job day keep his side chick away -sasha grey,-0.050000,0.100000,0.000


In [6]:
train_df= caption_df[["train" in val for val in caption_df["sets"]]]
dev_seen_df= caption_df[["dev_seen" in val for val in caption_df["sets"]]]
test_seen_df= caption_df[["test_seen" in val for val in caption_df["sets"]]]

In [7]:
# Create a Vectorizer Object
def vectorizer(dataframe, column, vocab=None):
    document= list(dataframe[column])
    if(vocab==None):
        vectorizer = TfidfVectorizer()
    else:
        vectorizer = TfidfVectorizer(vocabulary=vocab)
    vectorizer.fit(document)
    vocab= vectorizer.vocabulary_
    vector = vectorizer.transform(document)
    return vector, vocab

def make_numpy_matrix(vector, dataframe):
    vectorised= vector.toarray()
    tempdf=[]
    for i in range(len(dataframe)):
        temp= np.append(vectorised[i], [dataframe.iloc[i]["polarity"], dataframe.iloc[i]["subjectivity"], dataframe.iloc[i]["hate count"], 
                                        dataframe.iloc[i]["label"]])
        tempdf.append(temp)
    tempdf= np.array(tempdf)
    return tempdf[:,:-1], tempdf[:,-1]

def print_metrics(y_pred2, y_dev):
    print(f"accuracy: {metrics.accuracy_score(y_pred2, y_dev)}")
    print(f"f1 score: {metrics.f1_score(y_pred2, y_dev)}")
    print(f"AUROC: {metrics.roc_auc_score(y_pred2, y_dev)}")
    print(f"Recall: {metrics.recall_score(y_pred2, y_dev)}") 
    print(f"Precision: {metrics.precision_score(y_pred2, y_dev)}")

In [8]:
vec, voc = vectorizer(train_df, "text")
vec_dev, _ = vectorizer(dev_seen_df, "text", voc)
vec_test, _ = vectorizer(test_seen_df, "text", voc)

X_train, y_train = make_numpy_matrix(vec, train_df)
X_test, y_test = make_numpy_matrix(vec_test, test_seen_df)
X_dev, y_dev = make_numpy_matrix(vec_dev, dev_seen_df)

In [9]:
#Note that here we only take the words into account because by definiton, naive bayes models are designed to consider word collections. 
# Having additional features would not logically make sense. 
mnb = ComplementNB()
mnb.fit(X_train[:,:-3], y_train)
print("Test Seen")
y_pred= mnb.predict(X_test[:,:-3])
print_metrics(y_pred, y_test)

print("\nDev Seen")
y_pred2= mnb.predict(X_dev[:,:-3])
print_metrics(y_pred2, y_dev)

Test Seen
accuracy: 0.566
f1 score: 0.4166666666666667
AUROC: 0.5805872791370248
Recall: 0.610236220472441
Precision: 0.3163265306122449

Dev Seen
accuracy: 0.546
f1 score: 0.39466666666666667
AUROC: 0.5565356182795699
Recall: 0.578125
Precision: 0.29959514170040485


In [10]:
#We see a very low value of precision in an unbalanced dataset
mnb = MultinomialNB()
mnb.fit(X_train[:,:-3], y_train)
print("Test Seen")
y_pred= mnb.predict(X_test[:,:-3])
print_metrics(y_pred, y_test)

print("\nDev Seen")
y_pred2= mnb.predict(X_dev[:,:-3])
print_metrics(y_pred2, y_dev)

Test Seen
accuracy: 0.53
f1 score: 0.12962962962962962
AUROC: 0.6105263157894737
Recall: 0.7
Precision: 0.07142857142857142

Dev Seen
accuracy: 0.52
f1 score: 0.1366906474820144
AUROC: 0.5633812504298782
Recall: 0.6129032258064516
Precision: 0.07692307692307693


The following three cells are commented because they take around 10 minutes to run each, uncomment if necessary, but the findings have been reported above

In [11]:
# svm_classifier = SVC()
# svm_classifier.fit(X_train, y_train)
# y_pred= svm_classifier.predict(X_test)
# print_metrics(y_pred, y_test)

In [12]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# importances = rf.feature_importances_
# importances
# y_pred= rf.predict(X_test)
# print_metrics(y_pred, y_test)

In [13]:
# selfr=2000
# indices = np.argsort(importances)[::-1]
# selected_features = X_train[:, indices[:selfr]]
# rf2 = RandomForestClassifier(n_estimators=1000)
# rf2.fit(selected_features, y_train)
# temp2= X_test[:, indices[:selfr]]
# y_pred2= rf2.predict(temp2)
# print_metrics(y_pred2, y_test)

We shall now undersample the majority class, ie the non hateful memes, and then check our results

In [14]:
class_0_data = train_df[train_df['label'] == 0]
class_1_data = train_df[train_df['label'] == 1]
undersampled_class_0 = class_0_data.sample(n=3019, random_state=42) #there are 3019 label 1 rows
balanced_df = pd.concat([undersampled_class_0, class_1_data])
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
vec2, voc2 = vectorizer(balanced_df, "text")
vec_dev2, _ = vectorizer(dev_seen_df, "text", voc2)
vec_test2, _ = vectorizer(test_seen_df, "text", voc2)

X_train2, y_train2 = make_numpy_matrix(vec2, balanced_df)
X_test2, y_test2 = make_numpy_matrix(vec_test2, test_seen_df)
X_dev2, y_dev2 = make_numpy_matrix(vec_dev2, dev_seen_df)

In [16]:
#We dont see a very significant difference from before because Complement Naive Bayes has been designed for imbalanced datasets
mnb2 = ComplementNB()
mnb2.fit(X_train2[:,:-3], y_train2)
print("Test Seen")
y_pred2= mnb2.predict(X_test2[:,:-3])
print_metrics(y_pred2, y_test2)

print("\nDev Seen")
y_pred_t= mnb2.predict(X_dev2[:,:-3])
print_metrics(y_pred_t, y_dev2)

Test Seen
accuracy: 0.593
f1 score: 0.5901309164149043
AUROC: 0.5930633502806101
Recall: 0.5825049701789264
Precision: 0.5979591836734693

Dev Seen
accuracy: 0.592
f1 score: 0.6091954022988506
AUROC: 0.5935353535353535
Recall: 0.5781818181818181
Precision: 0.6437246963562753


In [17]:
#As opposed to earlier though, we see a great improvement in precision
mnb2 = MultinomialNB()
mnb2.fit(X_train2[:,:-3], y_train2)
print("Test Seen")
y_pred2= mnb2.predict(X_test2[:,:-3])
print_metrics(y_pred2, y_test2)

print("\nDev Seen")
y_pred_t= mnb2.predict(X_dev2[:,:-3])
print_metrics(y_pred_t, y_dev2)

Test Seen
accuracy: 0.593
f1 score: 0.5901309164149043
AUROC: 0.5930633502806101
Recall: 0.5825049701789264
Precision: 0.5979591836734693

Dev Seen
accuracy: 0.592
f1 score: 0.6091954022988506
AUROC: 0.5935353535353535
Recall: 0.5781818181818181
Precision: 0.6437246963562753


In [18]:
lrcf = LogisticRegression()
lrcf.fit(X_train2, y_train2)
print("Test Seen")
y_pred2= lrcf.predict(X_test2)
print_metrics(y_pred2, y_test2)

print("\nDev Seen")
y_pred2= lrcf.predict(X_dev2)
print_metrics(y_pred2, y_dev2)

Test Seen
accuracy: 0.59
f1 score: 0.5308924485125858
AUROC: 0.5926677489177489
Recall: 0.6041666666666666
Precision: 0.47346938775510206

Dev Seen
accuracy: 0.592
f1 score: 0.5486725663716814
AUROC: 0.5939644481190575
Recall: 0.6048780487804878
Precision: 0.5020242914979757


In [19]:
#As we can see, when we use the original imbalanced dataset is giving poor results
lrcf = LogisticRegression()
lrcf.fit(X_train, y_train)
print("Test Seen")
y_pred= lrcf.predict(X_test)
print_metrics(y_pred, y_test)

print("\nDev Seen")
y_pred= lrcf.predict(X_dev)
print_metrics(y_pred, y_dev)

Test Seen
accuracy: 0.561
f1 score: 0.30866141732283464
AUROC: 0.6086912684008873
Recall: 0.6758620689655173
Precision: 0.2

Dev Seen
accuracy: 0.538
f1 score: 0.28923076923076924
AUROC: 0.5643152266375016
Recall: 0.6025641025641025
Precision: 0.1902834008097166


We can now also try oversampling the minority class to see if there are any changes

In [20]:
class_0_data = train_df[train_df['label'] == 0]
class_1_data = train_df[train_df['label'] == 1]
difference= len(class_0_data)-len(class_1_data)
oversampled_class_1 = class_1_data.sample(n=difference, random_state=42) 
balanced_df2 = pd.concat([class_0_data, class_1_data, oversampled_class_1])
balanced_df2 = balanced_df2.sample(frac=1, random_state=42).reset_index(drop=True)

In [21]:
vec3, voc3 = vectorizer(balanced_df2, "text")
vec_dev3, _ = vectorizer(dev_seen_df, "text", voc3)
vec_test3, _ = vectorizer(test_seen_df, "text", voc3)

X_train3, y_train3 = make_numpy_matrix(vec3, balanced_df2)
X_test3, y_test3 = make_numpy_matrix(vec_test3, test_seen_df)
X_dev3, y_dev3 = make_numpy_matrix(vec_dev3, dev_seen_df)

In [22]:
#There is a slight decrease in performance, but that is expected, because we are repeating information, bringing in redundancy
#Yet, even in this case, all the metrics are above 0.5 indicating some information or pattern has been learnt
mnb3 = MultinomialNB()
mnb3.fit(X_train3[:,:-3], y_train3)
print("Test Seen")
y_pred3= mnb3.predict(X_test3[:,:-3])
print_metrics(y_pred3, y_test3)

print("\nDev Seen")
y_pred_t3= mnb3.predict(X_dev3[:,:-3])
print_metrics(y_pred_t3, y_dev3)

Test Seen
accuracy: 0.574
f1 score: 0.5571725571725572
AUROC: 0.5736710323574731
Recall: 0.5677966101694916
Precision: 0.5469387755102041

Dev Seen
accuracy: 0.578
f1 score: 0.5667351129363449
AUROC: 0.5778846153846153
Recall: 0.575
Precision: 0.5587044534412956


In [23]:
mnb3 = ComplementNB()
mnb3.fit(X_train3[:,:-3], y_train3)
print("Test Seen")
y_pred3= mnb3.predict(X_test3[:,:-3])
print_metrics(y_pred3, y_test3)

print("\nDev Seen")
y_pred_t3= mnb3.predict(X_dev3[:,:-3])
print_metrics(y_pred_t3, y_dev3)

Test Seen
accuracy: 0.574
f1 score: 0.5571725571725572
AUROC: 0.5736710323574731
Recall: 0.5677966101694916
Precision: 0.5469387755102041

Dev Seen
accuracy: 0.578
f1 score: 0.5667351129363449
AUROC: 0.5778846153846153
Recall: 0.575
Precision: 0.5587044534412956


In [24]:
#We clearly see a drop in performance here as well, reinforcing our idea that undersampling is more suited in our problem
clf_lr2 = LogisticRegression()
clf_lr2.fit(X_train3, y_train3)
print("Test Seen")
y_pred3= clf_lr2.predict(X_test3)
print_metrics(y_pred3, y_test3)

print("\nDev Seen")
y_pred_t3= clf_lr2.predict(X_dev3)
print_metrics(y_pred_t3, y_dev3)

Test Seen
accuracy: 0.599
f1 score: 0.52989449003517
AUROC: 0.6040734157617274
Recall: 0.6225895316804407
Precision: 0.46122448979591835

Dev Seen
accuracy: 0.592
f1 score: 0.5405405405405406
AUROC: 0.5949975708230721
Recall: 0.6091370558375635
Precision: 0.48582995951417
